### Twilight Struggle

In [1]:
import numpy as np
import random

from twilight_map import *
from game_mechanics import *
from twilight_cards import *

### Game Instance

In [2]:
# this is a temporary measure. It should go into UI.new()
g = Game()
g.ui.run()

Initalising game..

----- USSR Player: -----
You may modify 6 influence in these countries. Type in their country indices, separated by commas (no spaces).
Yugoslavia, 20
Czechoslovakia, 18
Bulgaria, 22
Austria, 14
Hungary, 19
Romania, 21
East_Germany, 16
Poland, 17
Finland, 7
> q

----- US Player: -----
You may modify 7 influence in these countries. Type in their country indices, separated by commas (no spaces).
Denmark, 8
Austria, 14
Canada, 3
Sweden, 6
Spain_Portugal, 11
Turkey, 23
Italy, 12
Benelux, 9
Norway, 5
UK, 4
West_Germany, 15
Greece, 13
France, 10
Finland, 7
> q

----- US Player: -----
You may modify 2 influence in these countries. Type in their country indices, separated by commas (no spaces).
US, 2
Canada, 3
UK, 4
Israel, 26
Iran, 30
Australia, 42
Philippines, 44
Japan, 45
South_Korea, 47
South_Africa, 59
Panama, 72
> q
USSR Victory!
Final scoring complete.
USSR Victory!
Final scoring complete.
USSR Victory!
Final scoring complete.
USSR Victory!
Final scoring complete.
US

In [3]:
g.choose_headline(Side.USSR)


----- USSR Player: -----
You may headline any of these cards. Type in the card index.
Asia_Scoring, 1
Containment, 25
Marshall_Plan, 23
Defectors, 103
Indo_Pakistani_War, 24
The_Cambridge_Five, 104
Decolonization, 30
Middle_East_Scoring, 3
U2_Incident, 60
> 1


In [4]:
g.ussr_hand

[The_China_Card - 4,
 Containment - 3,
 Marshall_Plan - 4,
 Defectors - 2,
 Indo_Pakistani_War - 2,
 The_Cambridge_Five - 2,
 Decolonization - 2,
 Middle_East_Scoring,
 U2_Incident - 3]